Copyright 2021-2023 Lawrence Livermore National Security, LLC and other MuyGPyS
Project Developers. See the top-level COPYRIGHT file for details.

SPDX-License-Identifier: MIT

# Anisotropic tutorial with optimization loop chassis

This notebook walks through the [Anisotropic Metric Tutorial](../docs/examples/anisotropic_tutorial.ipynb) using experimental optimaztion loop chassis. The goal is to recover the response on the held-out test data by training a simple anisotropic `MuyGPS` model on the perturbed training data with `nu` smoothness hyperparameter known, while the two `distance scaling` hyperparameters are to be learned.
1. Sample a 2D surface from a conventional GP
2. Construct nearest neighbor lookups
3. Create duplicate of `MuyGPyS` object used in `sampler` to be used for training and inference
4. Call `optimize_from_tensors_mini_batch` to sample batches of data, construct tensors, and run bayes optimization using numpy math backend

In [ ]:
import sys
for m in sys.modules.keys():
    if m.startswith("Muy"):
        sys.modules.pop(m)
%env MUYGPYS_BACKEND=numpy
%env MUYGPYS_FTYPE=64

import numpy as np

from docs.examples.utils import UnivariateSampler2D, print_results

from MuyGPyS.gp import MuyGPS
from MuyGPyS.gp.distortion import AnisotropicDistortion, IsotropicDistortion, l2
from MuyGPyS.gp.hyperparameter import ScalarHyperparameter
from MuyGPyS.gp.kernels import Matern
from MuyGPyS.gp.noise import HomoscedasticNoise
from MuyGPyS.gp.tensors import make_predict_tensors
from MuyGPyS.neighbors import NN_Wrapper
from MuyGPyS.optimize.batch import sample_batch
from MuyGPyS.optimize.experimental.chassis import optimize_from_tensors_mini_batch
from MuyGPyS.optimize.sigma_sq import muygps_sigma_sq_optim

In [ ]:
np.random.seed(0)
points_per_dim = 60                             # Observations per dimension
train_step = 13                                 # Train/test data split
nugget_noise = HomoscedasticNoise(1e-14)        # Assume no noise in truth
measurement_noise = HomoscedasticNoise(1e-7)    # Noise to perturb train
sim_nu = ScalarHyperparameter(1.5)              # HP smoothness
sim_length_scale0 = ScalarHyperparameter(0.1)   # HP distance scaling dim 0
sim_length_scale1 = ScalarHyperparameter(0.5)   # HP distance scaling dim 1
sampler = UnivariateSampler2D(
    points_per_dim=points_per_dim,
    train_step=train_step,
    kernel=Matern(
        nu=sim_nu,
        metric=AnisotropicDistortion(
            l2,
            length_scale0=sim_length_scale0,
            length_scale1=sim_length_scale1,
        ),
    ),
    eps=nugget_noise,
    measurement_eps=measurement_noise,
)
train_features, test_features = sampler.features()
train_responses, test_responses = sampler.sample()
sampler.plot_sample()

nn_count = 30
#TODO apply anistropic distance function using learned distance scaling parameters
nbrs_lookup = NN_Wrapper(train_features, nn_count, nn_method="exact", algorithm="ball_tree")

exp_length_scale0 = ScalarHyperparameter("log_sample", (0.01, 1.0))
exp_length_scale1 = ScalarHyperparameter("log_sample", (0.01, 1.0))
muygps = MuyGPS(
    kernel=Matern(
        nu=sim_nu,
        metric=AnisotropicDistortion(
            l2,
            length_scale0=exp_length_scale0,
            length_scale1=exp_length_scale1,
        ),
    ),
    eps=measurement_noise,
)


Validate using all training data.

In [ ]:
muygps = optimize_from_tensors_mini_batch(
    muygps,
    train_features,
    train_responses,
    nbrs_lookup,
    batch_count=sampler.train_count,
    train_count=sampler.train_count,
    num_epochs=1,
    batch_features=None,
    loss_method="lool",
    obj_method="loo_crossval",
    sigma_method="analytic",
    verbose=True,
    random_state=1,
    init_points=5,
    n_iter=20,
    allow_duplicate_points=True,
)
print(f"BayesianOptimization finds an optimimal pair of length scales")
print(f"\tlength_scale0 is {muygps.kernel.distortion_fn.length_scale['length_scale0']()}")
print(f"\tlength_scale1 is {muygps.kernel.distortion_fn.length_scale['length_scale1']()}")
print(f"Optimized anisotropic sigma_sq: {muygps.sigma_sq()}")


Test optimization loop chassis using sample batch size = 100.

In [ ]:
batch_count=55
train_count=sampler.train_count
num_epochs=int(sampler.train_count / batch_count)
muygps = optimize_from_tensors_mini_batch(
    muygps,
    train_features,
    train_responses,
    nbrs_lookup,
    batch_count=batch_count,
    train_count=train_count,
    num_epochs=num_epochs,
    batch_features=None,
    loss_method="lool",
    obj_method="loo_crossval",
    sigma_method="analytic",
    verbose=True,
    random_state=1,
    init_points=5,
    n_iter=20,
)
print(f"BayesianOptimization finds an optimimal pair of length scales")
print(f"\tlength_scale0 is {muygps.kernel.distortion_fn.length_scale['length_scale0']()}")
print(f"\tlength_scale1 is {muygps.kernel.distortion_fn.length_scale['length_scale1']()}")
print(f"Optimized anisotropic sigma_sq: {muygps.sigma_sq()}")


Predict the response of test data.
1. Find the indices of the nearest neighbors of test elements
2. Make distance difference and target tensors for test data
3. Create kernel tensors and find posterior means and variances associated with each training prediction
4. Evaluate prediction performance in terms of RMSE, mean diagonal posterior variance, the mean 95% confidence interval size, and the coverage, which ideally should be near 95%. 